Check that the FBA model overlaps with essentiality data.

In [3]:
import sys
sys.path.append('../flux_balance_analysis')
from scipy import stats
import cobra
from single_knockouts import single_knockout_loss_costs , single_knockout_modified_loss_cost
from double_knockouts import load_sd_minus_his

In [4]:
model = load_sd_minus_his('../data/external/yeast_7.6/yeast_7.6.xml')
with open('../data/processed/Essential_ORFs.txt', 'r') as f:
    essentialGenes = set([l.strip() for l in f.readlines()])

cobra/core/Reaction.py:108 UserWarning: uppercase AND/OR found in rule '((YDL174C AND YEL039C) OR (YDL174C AND YJR048W) OR (YEL039C AND YEL071W) OR (YEL071W AND YJR048W))' for '<Reaction r_0001 at 0x1155b1f50>'
cobra/core/Reaction.py:108 UserWarning: uppercase AND/OR found in rule '((YDL178W AND YEL039C) OR (YDL178W AND YJR048W))' for '<Reaction r_0002 at 0x1155b1f10>'
cobra/core/Reaction.py:108 UserWarning: uppercase AND/OR found in rule '((YEL039C AND YML054C) OR (YJR048W AND YML054C))' for '<Reaction r_0004 at 0x1155b1fd0>'
cobra/core/Reaction.py:108 UserWarning: uppercase AND/OR found in rule '(YGR032W OR YLR342W)' for '<Reaction r_0005 at 0x1155b1e10>'
cobra/core/Reaction.py:108 UserWarning: uppercase AND/OR found in rule '(YGR143W OR YPR159W)' for '<Reaction r_0006 at 0x1155b1f90>'
cobra/core/Reaction.py:108 UserWarning: uppercase AND/OR found in rule '(YEL038W AND YMR009W)' for '<Reaction r_0013 at 0x1155b5650>'
cobra/core/Reaction.py:108 UserWarning: uppercase AND/OR found in r

In [ ]:
print 'using gene-loss costs'
print len(essentialGenes), 'experimental essential genes'
genes = set([g.id for g in model.genes])
print len(genes), 'genes in model'
print len(genes.intersection(essentialGenes)), 'overlap'
_wtGrew, glc, flc = single_knockout_loss_costs(model)
modifiedCost = single_knockout_modified_loss_cost(model)

In [6]:
print 'using gene-loss costs'
print len(essentialGenes), 'experimental essential genes'
genes = set([g.id for g in model.genes])
print len(genes), 'genes in model'
print len(genes.intersection(essentialGenes)), 'overlap'
_wtGrew, glc, flc = single_knockout_loss_costs(model)
modifiedCost = single_knockout_modified_loss_cost(model)

def essential_gene_prediction_hypothesis_test(costs):
    modelPredictions = set([orfID for orfID, cost in costs.items() if cost > 0.999])
    print len(modelPredictions), 'predicted essential'
    tp = len(modelPredictions.intersection(essentialGenes))
    print tp, 'predicted correctly'
    fp = len(modelPredictions) - tp
    fn = len(genes.intersection(essentialGenes)) - tp
    tn = len(genes) - (tp + fp + fn)
    odds, pval = stats.fisher_exact([[tn, fp],[fn, tp]])
    print 'p-value %.1e' % pval
    print 'odds-ratio %.1f' % odds
    
    
print '\nGene-loss'
essential_gene_prediction_hypothesis_test(glc)
print '\nFunction-loss'
essential_gene_prediction_hypothesis_test(flc)
print '\nGene-loss with non-redundant isoenzymes'
essential_gene_prediction_hypothesis_test(modifiedCost)

using gene-loss costs
1122 experimental essential genes
909 genes in model
139 overlap

Gene-loss
108 predicted essential
77 predicted correctly
p-value 1.3e-47
odds-ratio 29.6

Function-loss
144 predicted essential
64 predicted correctly
p-value 4.7e-21
odds-ratio 7.4

Gene-loss with non-redundant isoenzymes
165 predicted essential
80 predicted correctly
p-value 2.0e-31
odds-ratio 10.9


In [20]:
glcCorrect = glcPredictions.intersection(essentialGenes)
flcCorrect = flcPredictions.intersection(essentialGenes)
print glcCorrect.difference(flcCorrect)
print flcCorrect.difference(glcCorrect)
print [len(model.genes.get_by_id(orfID).reactions) for orfID in glcCorrect.difference(flcCorrect)]
print [[r.gene_reaction_rule for r in model.genes.get_by_id(orfID).reactions] 
       for orfID in flcCorrect.difference(glcCorrect)]

set(['YKL004W', 'YDL015C', 'YBR029C', 'YKL182W', 'YGL055W', 'YML126C', 'YDR454C', 'YMR208W', 'YDR367W', 'YOR236W', 'YPL028W', 'YPR113W', 'YGR175C', 'YGR094W', 'YKL024C', 'YPL231W'])
set(['YLR153C', 'YBR038W', 'YDR023W'])
[80, 7, 14, 2, 2, 2, 3, 4, 80, 2, 2, 8, 2, 2, 4, 2]
[['YLR153C', '(YAL054C or YLR153C)'], ['(YBR023C or YBR038W or YNL192W)'], ['(YDR023W or YHR011W)']]
